# Efficient Vaccine Distribution
### Delivering Excess Vaccine Supply to Most Needed Locations
by Kaan Karamente; Saif Ahmed; Matt Brown; Chad Meley (info@kinetica.com)

The CDC publishes extensive daily data on vaccine delivery, availability, and administration. From this, an approximate "excess" supply can be calculated at the state level for each state in the US. Further, from standard data on vaccine expirations, we can calculate excess supply which is at risk of expiring. *What if we could donate the excess supply to needy countries?* This would make good use of a sunk-cost as well as vaccinate more people globally, and reduce the risk of newer, more deadly variants which could harm us all.

How would we efficiently get the vaccines to the destinations given tight vaccine expiration timeframes? We can use commercial ariline networks! All data on airline networks and routes are public and we can create a graph from this.

#### Required Data
To achieve this logistical optimization, we'll need several datasets. The eight key datasets are listed below:

* Airlines, IATA Codes, and Callsigns
https://kinetica-community.s3.amazonaws.com/vaccine-distro/airlines.csv

* Airports, Countries serviced, and Geo-coordinates
https://kinetica-community.s3.amazonaws.com/vaccine-distro/airport-to-country-map.csv

* Airport IATA Codes
https://kinetica-community.s3.amazonaws.com/vaccine-distro/airports.csv

* US Vaccine Statistics -- Availability, Usage (to calculate excess)
https://kinetica-community.s3.amazonaws.com/vaccine-distro/vaccine-us.csv

* Map of Country ISO Alpha-2 to Alpha-3 for Cross-Dataset Mapping
https://kinetica-community.s3.amazonaws.com/vaccine-distro/map_iso_alpha2_alpha3.csv

* Global COVID Statistics (to calculate need/demand for vaccine) 
https://kinetica-community.s3.amazonaws.com/vaccine-distro/owid-covid-data.csv

* Airport-to-Airport Routes and Flight Map
https://kinetica-community.s3.amazonaws.com/vaccine-distro/routes.csv

* List of US Airports (to segment "excess supply" locations)
https://kinetica-community.s3.amazonaws.com/vaccine-distro/us-airports.csv    

We are now going to load a number of files into the database, but first you'll need to get them onto the database machine. Grab all the files referenced above and either put them into the /mnt/persist folder of Kinetica Dev Edition, or load them into a directory of your choice (e.g., /vaccine) on Kinetica Cloud via the KIFS upload screen. You can find more information here: https://docs.kinetica.com/7.1/tools/kifs/

We took all the files above and loaded them onto the Kinetica file system under the /vaccine folder:
* kifs://vaccine/airlines.csv
* kifs://vaccine/airport-to-country-map.csv
* kifs://vaccine/airports.csv
* kifs://vaccine/vaccine-us.csv
* kifs://vaccine/map_iso_alpha2_alpha3.csv
* kifs://vaccine/owid-covid-data.csv
* kifs://vaccine/routes.csv
* kifs://vaccine/us-airports.csv

Both the supply and demand datasets are highly dynamic, changing at least once a day. The above dataset will provide you a year's worth of data, but if you wish to run an as-of-now optmization, it is best to ingest current vaccine Supply and Demand from Kinetica Confluent Cloud topics:
* Broker: pkc-ep9mm.us-east-2.aws.confluent.cloud:9092
* Topics: vaccine-distro-us-states; vaccine-distro-countries-global

In [21]:
import os
import csv
import gpudb

We'll be interacting with Kinetica along the way, loading data, setting up graph optimziations, and viewing results. Ensure to export these environment variables (or override them below): **KINETICA_HOST, KINETICA_USER, KINETICA_PASS**

In [22]:
KINETICA_HOST = os.getenv('KINETICA_HOST', "localhost:9191")
KINETICA_USER = os.getenv('KINETICA_USER', "admin")
KINETICA_PASS = os.getenv('KINETICA_PASS')
db = gpudb.GPUdb(host=KINETICA_HOST, username=KINETICA_USER, password=KINETICA_PASS)              

First, we'll take the airports and airline routes data above and create a graph from it. More information about creating graphs is available at https://docs.kinetica.com/7.1/graph_solver/network_graph_solver/. Below we work through it step-by-step. Start by downloading the airports and routes file referenced above (we put it into a "./data" directory.) Our goal will be to create nodes and edges files so we can create a graph which will drive logistics.

In [23]:
INPUT_FILE_AIRPORTS = "data_originals/airports.csv"
INPUT_FILE_ROUTES = "data_originals/routes.csv"

OUTPUT_FILE_NODES = "out_nodes.csv"
OUTPUT_FILE_EDGES = "out_edges.csv"

FIELDS_NODES = ["NODE_ID",
                "NODE_X",
                "NODE_Y",
                "NODE_NAME",
                "NODE_WKTPOINT",
                "NODE_LABEL",
                "IATA",
                "ICAO",
                "CITY",
                "COUNTRY"]

FIELDS_EDGES = ["EDGE_ID",
                "EDGE_NODE1_ID",
                "EDGE_NODE2_ID",
                "EDGE_WKTLINE",
                "EDGE_NODE1_X",
                "EDGE_NODE1_Y",
                "EDGE_NODE2_X",
                "EDGE_NODE2_Y",
                "EDGE_NODE1_WKTPOINT",
                "EDGE_NODE2_WKTPOINT",
                "EDGE_NODE1_NAME",
                "EDGE_NODE2_NAME",
                "EDGE_DIRECTION",
                "EDGE_LABEL",
                "EDGE_WEIGHT_VALUESPECIFIED"]

In [24]:
# Helper function to simplify strings
def cleanse(in_str):
    out_str = in_str
    out_str.replace(",", "")
    out_str.replace("'", "")
    return out_str

# TODO: this is just a rough measure, a stand-in for now
# https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
def rough_distance(slon, slat, dlon, dlat):
    from math import sin, cos, sqrt, atan2, radians
    # approximate radius of earth in km
    R = 6373.0
    lat1 = radians(float(slat))
    lon1 = radians(float(slon))
    lat2 = radians(float(dlat))
    lon2 = radians(float(dlon))
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance    

In [25]:
lookup_airport = {}
airport_nodes = []

## ------------------------------------------------------------------------
## Create Lookup table for Edge Enrichment

input_file = csv.DictReader(open(INPUT_FILE_AIRPORTS))
for row in input_file:
    lookup_airport[row['AIRPORT_ID']] = {
        'AIRPORT_ID': row['AIRPORT_ID'],
        'NAME': cleanse(row['NAME']),
        'CITY': cleanse(row['CITY']),
        'COUNTRY': cleanse(row['COUNTRY']),
        'IATA': row['IATA'],
        'ICAO': row['ICAO'],
        'LATITUDE': row['LATITUDE'],
        'LONGITUDE': row['LONGITUDE']
    }

    ## ------------------------------------------------------------------------
    ## Create Nodes

    if row['AIRPORT_ID'] == "\\N":
        continue
    nlon = row['LONGITUDE']
    nlat = row['LATITUDE']
    if row['IATA']=="\\N":
        row['IATA']=None
        nodename = f"{row['ICAO']}: {cleanse(row['NAME'])}"
        nodelabel = f"{row['ICAO']}: {cleanse(row['NAME'])}; {cleanse(row['CITY'])}, {cleanse(row['COUNTRY'])}"
    else:
        nodename = f"{row['IATA']}: {cleanse(row['NAME'])}"
        nodelabel = f"{row['IATA']}: {cleanse(row['NAME'])}; {cleanse(row['CITY'])}, {cleanse(row['COUNTRY'])}"

    persistable = {
        "NODE_ID": row['AIRPORT_ID'],
        "NODE_X": nlon,
        "NODE_Y": nlat,
        "NODE_NAME": nodename,
        "NODE_WKTPOINT": f"POINT({nlon} {nlat})",
        "NODE_LABEL": nodelabel,
        "IATA": row['IATA'],
        "ICAO": row['ICAO'],
        "CITY": cleanse(row['CITY']),
        "COUNTRY": cleanse(row['COUNTRY'])
    }
    airport_nodes.append(persistable)
    #print(f"Adding node {persistable['NODE_LABEL']}")

#print(f"Writing {len(airport_nodes)} rows")

with open(OUTPUT_FILE_NODES, 'w', newline='\n') as csvfile:        
    writer = csv.DictWriter(csvfile, fieldnames=FIELDS_NODES)
    writer.writeheader()
    for n in airport_nodes:
        writer.writerow(n)

## ------------------------------------------------------------------------
## Create Edges

inter_airport_network_edges = []
edge_id = 10000

input_file = csv.DictReader(open(INPUT_FILE_ROUTES))
for row in input_file:
    edge_id = edge_id + 1
    if row['SOURCE_AIRPORT_ID'] == "\\N":
        #print(f"Warn source airport {row['SOURCE_AIRPORT_ID']} is Null, skipping...")
        continue
    if row['DEST_AIRPORT_ID'] == "\\N":
        #print(f"Warn source airport {row['DEST_AIRPORT_ID']} is Null, skipping...")
        continue
    if str(row['SOURCE_AIRPORT_ID']) not in lookup_airport:
        #print(f"Warn source airport {row['SOURCE_AIRPORT_ID']} not found in Airports lookup table, skipping...")
        continue
    if str(row['DEST_AIRPORT_ID']) not in lookup_airport:
        #print(f"Warn destination airport {row['SOURCE_AIRPORT_ID']} not found in Airports lookup table, skipping...")
        continue
    if lookup_airport[row['SOURCE_AIRPORT_ID']]['COUNTRY'] == lookup_airport[row['DEST_AIRPORT_ID']]['COUNTRY']:
        # skipping domestic flight
        continue
    slon = lookup_airport[row['SOURCE_AIRPORT_ID']]['LONGITUDE']
    slat = lookup_airport[row['SOURCE_AIRPORT_ID']]['LATITUDE']
    dlon = lookup_airport[row['DEST_AIRPORT_ID']]['LONGITUDE']
    dlat = lookup_airport[row['DEST_AIRPORT_ID']]['LATITUDE']
    persistable = {
        "EDGE_ID": edge_id,
        "EDGE_NODE1_ID": row['SOURCE_AIRPORT_ID'],
        "EDGE_NODE2_ID": row['DEST_AIRPORT_ID'],
        "EDGE_WKTLINE": f"LINESTRING({slon} {slat}, {dlon} {dlat})",
        "EDGE_NODE1_X": slon,
        "EDGE_NODE1_Y": slat,
        "EDGE_NODE2_X": dlon,
        "EDGE_NODE2_Y": dlat,
        "EDGE_NODE1_WKTPOINT": f"POINT({slon} {slat})",
        "EDGE_NODE2_WKTPOINT": f"POINT({dlon} {dlat})",
        "EDGE_NODE1_NAME": f"{lookup_airport[row['SOURCE_AIRPORT_ID']]['IATA']}: {lookup_airport[row['SOURCE_AIRPORT_ID']]['NAME']}",
        "EDGE_NODE2_NAME": f"{lookup_airport[row['DEST_AIRPORT_ID']]['IATA']}: {lookup_airport[row['DEST_AIRPORT_ID']]['NAME']}",
        "EDGE_DIRECTION": "0",
        "EDGE_LABEL": f"'{row['AIRLINE']} {row['AIRLINE_ID']} from {lookup_airport[row['SOURCE_AIRPORT_ID']]['IATA']} --> {lookup_airport[row['DEST_AIRPORT_ID']]['IATA']}'",
        "EDGE_WEIGHT_VALUESPECIFIED": rough_distance(slon, slat, dlon, dlat)
    }
    inter_airport_network_edges.append(persistable)
    #print(f"Adding edge {persistable['EDGE_LABEL']}")

print(f"Writing {len(inter_airport_network_edges)} rows")

with open(OUTPUT_FILE_EDGES, 'w', newline='\n') as csvfile:        
    writer = csv.DictWriter(csvfile, fieldnames=FIELDS_EDGES)
    writer.writeheader()
    for i in inter_airport_network_edges:
        writer.writerow(i)

Writing 34710 rows


In real life, data is always dirty, especially when joining data from different sources. While the above code looks extensive, we're just throwing out bad (e.g., airline routes mismatching the airport list, airports or routes with null data, etc.) At the end of the process, we end up with graph nodes and edges, ready to load into Kinetica:

     -rw-r--r--   1 saif  staff   1397234 Aug  2 10:30 out_nodes.csv
     -rw-r--r--   1 saif  staff  11438747 Aug  2 10:30 out_edges.csv


Now that we have the graph set up, we'll also need supply and demand figures. Of course, "supply" and "demand" are subjective figures -- we will need to derive these figures from high-order public data and personal judgements on which health data should drive demand (positivity rate, age distribution, vaccination rate, deaths, etc.) We present a simple approach but encourage the community consider this deeply and propose better or more fair mechanisms to calculate supply/demand. Lets start by loading the raw data:

Lets create a schema to keep all our work in one place:

In [40]:
exec_result = db.execute_sql("CREATE SCHEMA DEMO_Vaccine_Distro;")
#print(exec_result)

Lets load in the nodes and edges files we just created.

In [41]:
db.execute_sql("""
    CREATE OR REPLACE MATERIALIZED EXTERNAL TABLE "DEMO_Vaccine_Distro"."airport_nodes"
    (
       "NODE_ID" SMALLINT NOT NULL,
       "NODE_X" DOUBLE NOT NULL,
       "NODE_Y" DOUBLE NOT NULL,
       "NODE_NAME" VARCHAR (128) NOT NULL,
       "NODE_WKTPOINT" GEOMETRY NOT NULL,
       "NODE_LABEL" VARCHAR (128) NOT NULL,
       "IATA" VARCHAR (4) NOT NULL,
       "ICAO" VARCHAR (4) NOT NULL,
       "CITY" VARCHAR (64),
       "COUNTRY" VARCHAR (32, dict) NOT NULL
    )
    FILE PATHS 'kifs://vaccine/out_nodes.csv'
    FORMAT DELIMITED TEXT;
    """)

{'count_affected': 6072,
 'response_schema_str': '{"type":"record","name":"generic_response","fields":[{"name":"column_1","type":{"type":"array","items":"string"}},{"name":"column_headers","type":{"type":"array","items":"string"}},{"name":"column_datatypes","type":{"type":"array","items":"string"}}]}',
 'binary_encoded_response': b'\x00\x02\ndummy\x00\x02\x0cstring\x00',
 'json_encoded_response': '',
 'total_number_of_records': -1,
 'has_more_records': False,
 'paging_table': '',
 'info': {'WARNING_0': '1626 rows were skipped while loading data.',
  'X-Kinetica-Group': 'DDL',
  'count': '6072'},
 'status_info': {'status': 'OK',
  'message': '',
  'data_type': 'execute_sql_response',
  'response_time': 0.76907},
 'record_type': kinetica.protocol.RecordType('', [('dummy', 'string')])}

In [37]:
db.execute_sql("""
    CREATE or REPLACE MATERIALIZED EXTERNAL TABLE "DEMO_Vaccine_Distro"."airport_routes"
    (
       "EDGE_ID" INTEGER NOT NULL,
       "EDGE_NODE1_ID" INTEGER NOT NULL,
       "EDGE_NODE2_ID" INTEGER NOT NULL,
       "EDGE_WKTLINE" GEOMETRY NOT NULL,
       "EDGE_NODE1_X" DOUBLE NOT NULL,
       "EDGE_NODE1_Y" DOUBLE NOT NULL,
       "EDGE_NODE2_X" DOUBLE NOT NULL,
       "EDGE_NODE2_Y" DOUBLE NOT NULL,
       "EDGE_NODE1_WKTPOINT" GEOMETRY NOT NULL,
       "EDGE_NODE2_WKTPOINT" GEOMETRY NOT NULL,
       "EDGE_NODE1_NAME" VARCHAR (128, dict) NOT NULL,
       "EDGE_NODE2_NAME" VARCHAR (128, dict) NOT NULL,
       "EDGE_DIRECTION" INTEGER NOT NULL,
       "EDGE_LABEL" VARCHAR (32) NOT NULL,
       "EDGE_WEIGHT_VALUESPECIFIED" DOUBLE NOT NULL
    )
    FILE PATHS 'kifs://vaccine/out_edges.csv'
    FORMAT DELIMITED TEXT;
    """)

{'count_affected': 34710,
 'response_schema_str': '{"type":"record","name":"generic_response","fields":[{"name":"column_1","type":{"type":"array","items":"string"}},{"name":"column_headers","type":{"type":"array","items":"string"}},{"name":"column_datatypes","type":{"type":"array","items":"string"}}]}',
 'binary_encoded_response': b'\x00\x02\ndummy\x00\x02\x0cstring\x00',
 'json_encoded_response': '',
 'total_number_of_records': -1,
 'has_more_records': False,
 'paging_table': '',
 'info': {'X-Kinetica-Group': 'DDL', 'count': '34710'},
 'status_info': {'status': 'OK',
  'message': '',
  'data_type': 'execute_sql_response',
  'response_time': 1.24642},
 'record_type': kinetica.protocol.RecordType('', [('dummy', 'string')])}

### Load Demand Data

Supply and Demand data needs to be loaded to drive the optimization. The data changes daily, so you can re-run the optimization daily for the most up-to-date logistical recommendations. To load the data, we recommend Kinetica External Tables. https://docs.kinetica.com/7.1/concepts/external_tables/

The External Tables funcitonality will allow easy loads, schema inference, and re-loads. Depending on the data source (which can sometimes be 'rough', inferred schemas occasionally require tweaks. Below we load the data using an inferred schema tweaked slighty. We encourage you to explore fully automated inference as well, if curious. The original data can be sourced from Our World in Data (https://ourworldindata.org/coronavirus) if desired.

In [42]:
db.execute_sql("""
    CREATE or REPLACE MATERIALIZED EXTERNAL TABLE "DEMO_Vaccine_Distro"."global_covid_statistics"
    (
       "iso_code" VARCHAR (8, dict) NOT NULL,
       "continent" VARCHAR (16, dict),
       "location" VARCHAR (32, dict) NOT NULL,
       "date" DATE (dict) NOT NULL,
       "total_cases" DECIMAL(18,4),
       "new_cases" DECIMAL(18,4),
       "new_cases_smoothed" DECIMAL(18,4),
       "total_deaths" DECIMAL(18,4),
       "new_deaths" DECIMAL(18,4),
       "new_deaths_smoothed" DECIMAL(18,4),
       "total_cases_per_million" DECIMAL(18,4),
       "new_cases_per_million" DECIMAL(18,4),
       "new_cases_smoothed_per_million" DECIMAL(18,4),
       "total_deaths_per_million" DECIMAL(18,4),
       "new_deaths_per_million" DECIMAL(18,4),
       "new_deaths_smoothed_per_million" DECIMAL(18,4),
       "reproduction_rate" DECIMAL(18,4),
       "icu_patients" DECIMAL(18,4),
       "icu_patients_per_million" DECIMAL(18,4),
       "hosp_patients" DECIMAL(18,4),
       "hosp_patients_per_million" DECIMAL(18,4),
       "weekly_icu_admissions" DECIMAL(18,4),
       "weekly_icu_admissions_per_million" DECIMAL(18,4),
       "weekly_hosp_admissions" DECIMAL(18,4),
       "weekly_hosp_admissions_per_million" DECIMAL(18,4),
       "new_tests" DECIMAL(18,4),
       "total_tests" DECIMAL(18,4),
       "total_tests_per_thousand" DECIMAL(18,4),
       "new_tests_per_thousand" DECIMAL(18,4),
       "new_tests_smoothed" DECIMAL(18,4),
       "new_tests_smoothed_per_thousand" DECIMAL(18,4),
       "positive_rate" DECIMAL(18,4),
       "tests_per_case" DECIMAL(18,4),
       "tests_units" VARCHAR (16, dict),
       "total_vaccinations" DECIMAL(18,4),
       "people_vaccinated" DECIMAL(18,4),
       "people_fully_vaccinated" DECIMAL(18,4),
       "new_vaccinations" DECIMAL(18,4),
       "new_vaccinations_smoothed" DECIMAL(18,4),
       "total_vaccinations_per_hundred" DECIMAL(18,4),
       "people_vaccinated_per_hundred" DECIMAL(18,4),
       "people_fully_vaccinated_per_hundred" DECIMAL(18,4),
       "new_vaccinations_smoothed_per_million" DECIMAL(18,4),
       "stringency_index" DECIMAL(18,4),
       "population" DECIMAL(18,4),
       "population_density" DECIMAL(18,4),
       "median_age" DECIMAL(18,4),
       "aged_65_older" DECIMAL(18,4),
       "aged_70_older" DECIMAL(18,4),
       "gdp_per_capita" DECIMAL(18,4),
       "extreme_poverty" DECIMAL(18,4),
       "cardiovasc_death_rate" DECIMAL(18,4),
       "diabetes_prevalence" DECIMAL(18,4),
       "female_smokers" DECIMAL(18,4),
       "male_smokers" DECIMAL(18,4),
       "handwashing_facilities" DECIMAL(18,4),
       "hospital_beds_per_thousand" DECIMAL(18,4),
       "life_expectancy" DECIMAL(18,4),
       "human_development_index" DECIMAL(18,4),
       "excess_mortality" DECIMAL(18,4)
    )
    FILE PATHS 'kifs://vaccine/owid-covid-data.csv'
    FORMAT DELIMITED TEXT;
    """)

{'count_affected': 76701,
 'response_schema_str': '{"type":"record","name":"generic_response","fields":[{"name":"column_1","type":{"type":"array","items":"string"}},{"name":"column_headers","type":{"type":"array","items":"string"}},{"name":"column_datatypes","type":{"type":"array","items":"string"}}]}',
 'binary_encoded_response': b'\x00\x02\ndummy\x00\x02\x0cstring\x00',
 'json_encoded_response': '',
 'total_number_of_records': -1,
 'has_more_records': False,
 'paging_table': '',
 'info': {'X-Kinetica-Group': 'DDL', 'count': '76701'},
 'status_info': {'status': 'OK',
  'message': '',
  'data_type': 'execute_sql_response',
  'response_time': 0.91694},
 'record_type': kinetica.protocol.RecordType('', [('dummy', 'string')])}

### Load Supply Data

Just like with demand data, we'll load supply data using External Tables. Fresh data can be sourced daily from https://covid.cdc.gov/covid-data-tracker/#vaccinations


In [31]:
db.execute_sql("""
    CREATE OR REPLACE MATERIALIZED EXTERNAL TABLE "DEMO_Vaccine_Distro"."vaccine_utilization_usa"
    (
       "State_Territory_Federal_Entity" VARCHAR (32),
       "Total_Doses_Delivered" INTEGER,
       "Doses_Delivered_per_100K" INTEGER,
       "18plus_Doses_Delivered_per_100K" INTEGER,
       "Total_Doses_Administered_by_State_where_Administered" INTEGER,
       "Doses_Administered_per_100k_by_State_where_Administered" INTEGER,
       "18plus_Doses_Administered_by_State_where_Administered" INTEGER,
       "18plus_Doses_Administered_per_100K_by_State_where_Administered" INTEGER,
       "People_with_at_least_One_Dose_by_State_of_Residence" INTEGER,
       "Percent_of_Total_Pop_with_at_least_One_Dose_by_State_of_Residence" DECIMAL(18,4),
       "People_18plus_with_at_least_One_Dose_by_State_of_Residence" INTEGER,
       "Percent_of_18plus_Pop_with_at_least_One_Dose_by_State_of_Residence" DECIMAL(18,4),
       "People_Fully_Vaccinated_by_State_of_Residence" INTEGER,
       "Percent_of_Total_Pop_Fully_Vaccinated_by_State_of_Residence" DECIMAL(18,4),
       "People_18plus_Fully_Vaccinated_by_State_of_Residence" INTEGER,
       "Percent_of_18plus_Pop_Fully_Vaccinated_by_State_of_Residence" DECIMAL(18,4),
       "Total_Number_of_Pfizer_doses_delivered" INTEGER,
       "Total_Number_of_Moderna_doses_delivered" INTEGER,
       "Total_Number_of_Janssen_doses_delivered" INTEGER,
       "Total_Number_of_doses_from_unknown_manufacturer_delivered" TINYINT,
       "Total_Number_of_Janssen_doses_administered" INTEGER,
       "Total_Number_of_Moderna_doses_administered" INTEGER,
       "Total_Number_of_Pfizer_doses_adminstered" INTEGER,
       "Total_Number_of_doses_from_unknown_manufacturer_administered" INTEGER,
       "People_Fully_Vaccinated_Moderna_Resident" INTEGER,
       "People_Fully_Vaccinated_Pfizer_Resident" INTEGER,
       "People_Fully_Vaccinated_Janssen_Resident" INTEGER,
       "People_Fully_Vaccinated_Unknown_2_dose_manufacturer_Resident" SMALLINT,
       "People_18plus_Fully_Vaccinated_Moderna_Resident" INTEGER,
       "People_18plus_Fully_Vaccinated_Pfizer_Resident" INTEGER,
       "People_18plus_Fully_Vaccinated_Janssen_Resident" INTEGER,
       "People_18plus_Fully_Vaccinated_Unknown_2_dose_manufacturer_Resident" SMALLINT,
       "People_with_2_Doses_by_State_of_Residence" INTEGER,
       "Percent_of_Total_Pop_with_1plus_Doses_by_State_of_Residence" DECIMAL(18,4),
       "People_18plus_with_1plus_Doses_by_State_of_Residence" INTEGER,
       "Percent_of_18plus_Pop_with_1plus_Doses_by_State_of_Residence" DECIMAL(18,4),
       "Percent_of_Total_Pop_with_2_Doses_by_State_of_Residence" DECIMAL(18,4),
       "People_18plus_with_2_Doses_by_State_of_Residence" INTEGER,
       "Percent_of_18plus_Pop_with_2_Doses_by_State_of_Residence" DECIMAL(18,4),
       "People_with_1plus_Doses_by_State_of_Residence" INTEGER,
       "People_65plus_with_at_least_One_Dose_by_State_of_Residence" INTEGER,
       "Percent_of_65plus_Pop_with_at_least_One_Dose_by_State_of_Residence" DECIMAL(18,4),
       "People_65plus_Fully_Vaccinated_by_State_of_Residence" INTEGER,
       "Percent_of_65plus_Pop_Fully_Vaccinated_by_State_of_Residence" DECIMAL(18,4),
       "People_65plus_Fully_Vaccinated_Moderna_Resident" INTEGER,
       "People_65plus_Fully_Vaccinated_Pfizer_Resident" INTEGER,
       "People_65plus_Fully_Vaccinated_Janssen_Resident" INTEGER,
       "People_65plus_Fully_Vaccinated_Unknown_2_dose_Manuf_Resident" SMALLINT,
       "65plus_Doses_Administered_by_State_where_Administered" INTEGER,
       "Doses_Administered_per_100k_of_65plus_pop_by_State_where_Administered" INTEGER,
       "Doses_Delivered_per_100k_of_65plus_pop" INTEGER,
       "People_12plus_with_at_least_One_Dose_by_State_of_Residence" INTEGER,
       "Percent_of_12plus_Pop_with_at_least_One_Dose_by_State_of_Residence" DECIMAL(18,4),
       "People_12plus_Fully_Vaccinated_by_State_of_Residence" INTEGER,
       "Percent_of_12plus_Pop_Fully_Vaccinated_by_State_of_Residence" DECIMAL(18,4),
       "People_12plus_Fully_Vaccinated_Moderna_Resident" INTEGER,
       "People_12plus_Fully_Vaccinated_Pfizer_Resident" INTEGER,
       "People_12plus_Fully_Vaccinated_Janssen_Resident" INTEGER,
       "People_12plus_Fully_Vaccinated_Unknown_2_dose_Manuf_Resident" SMALLINT,
       "12plus_Doses_Administered_by_State_where_Administered" INTEGER,
       "Doses_Administered_per_100k_of_12plus_pop_by_State_where_Administered" INTEGER,
       "Doses_Delivered_per_100k_of_12plus_pop" INTEGER
    )
    FILE PATHS 'kifs://vaccine/vaccine-us.csv'
    FORMAT DELIMITED TEXT;
    """)

{'count_affected': 63,
 'response_schema_str': '{"type":"record","name":"generic_response","fields":[{"name":"column_1","type":{"type":"array","items":"string"}},{"name":"column_headers","type":{"type":"array","items":"string"}},{"name":"column_datatypes","type":{"type":"array","items":"string"}}]}',
 'binary_encoded_response': b'\x00\x02\ndummy\x00\x02\x0cstring\x00',
 'json_encoded_response': '',
 'total_number_of_records': -1,
 'has_more_records': False,
 'paging_table': '',
 'info': {'X-Kinetica-Group': 'DDL', 'count': '63'},
 'status_info': {'status': 'OK',
  'message': '',
  'data_type': 'execute_sql_response',
  'response_time': 0.81596},
 'record_type': kinetica.protocol.RecordType('', [('dummy', 'string')])}

Once again, we present a simple approach to supply/demand -- but encourage the community consider this deeply and propose better or more fair mechanisms to calculate supply/demand. Your supply/demand mechanism just needs to output a supply and demand figure, source by source and destination by destination. We do this via two *Materialized Views.* We encourage you to read about these here: https://docs.kinetica.com/7.1/concepts/materialized_views/. They can be setup to automatically or periodically refresh based on underlying data changes, which is great for our use case.

### Supply

    CREATE OR REPLACE materialized VIEW DEMO_Vaccine_Distro.vaccine_supply_usa refresh ON change AS
    (
    select 
        Total_Doses_Delivered,
        Total_Doses_Administered_by_State_where_Administered,
        People_Fully_Vaccinated_by_State_of_Residence,
        People_18plus_Fully_Vaccinated_by_State_of_Residence,
        Total_Doses_Delivered - Total_Doses_Administered_by_State_where_Administered as CURRENT_VACCINE_INVENTORY,
        int(0.23 * (Total_Doses_Delivered - Total_Doses_Administered_by_State_where_Administered)) as "Donateable" /* PER BIDEN DONATION ESTIMATES */
    from DEMO_Vaccine_Distro.vaccine_utilization_usa
    ) 
    
### Demand

    CREATE OR REPLACE materialized VIEW DEMO_Vaccine_Distro.vaccine_demand_intl refresh ON change AS
    (
    SELECT iso_code,
                  location,
                  countrymap.ISO_ALPHA_2 as country_iso2,
                  countrymap.ISO_ALPHA_3 as country_iso3,
                  int(population) as "total_population",
                  int(people_fully_vaccinated) as "fully_vaccinated_population",
                  int(new_cases_per_million) as "new_cases_per_1mm",
                  int(total_deaths_per_million) as "total_deaths_per_1mm",
                  int(population-people_fully_vaccinated) AS "unvaccinated",
                  int(100*(population-people_fully_vaccinated)/population) AS "unvaccinated_pct",
                  (select int(avg(100*(population-people_fully_vaccinated)/population)) from DEMO_Vaccine_Distro.global_covid_statistics) as "avg_global_unvaccinated_pct",
                  if (people_fully_vaccinated , int(population-people_fully_vaccinated), int(0.92 * population) ) AS "vaccinate_doses_required"
           FROM   DEMO_Vaccine_Distro.global_covid_statistics gstat,
                  DEMO_Vaccine_Distro.map_iso_alpha23 countrymap
           WHERE  date='2021-06-01'
           AND    trim(countrymap.ISO_ALPHA_3) = trim(iso_code)

    ) 

In [43]:
db.execute_sql("""
    CREATE MATERIALIZED EXTERNAL TABLE "DEMO_Vaccine_Distro"."airport_metadata_usa"
    (
       "id" INTEGER,
       "ident" VARCHAR (8),
       "type" VARCHAR (16, dict),
       "name" VARCHAR (128),
       "latitude_deg" DOUBLE,
       "longitude_deg" DOUBLE,
       "elevation_ft" SMALLINT,
       "continent" VARCHAR (2),
       "country_name" VARCHAR (16, dict),
       "iso_country" VARCHAR (2),
       "region_name" VARCHAR (32, dict),
       "iso_region" VARCHAR (8, dict),
       "local_region" VARCHAR (2),
       "municipality" VARCHAR (64),
       "scheduled_service" TINYINT,
       "gps_code" VARCHAR (4),
       "iata_code" VARCHAR (4),
       "local_code" VARCHAR (8),
       "home_link" VARCHAR (128),
       "wikipedia_link" VARCHAR (128),
       "keywords" VARCHAR (256),
       "score" INTEGER,
       "last_updated" VARCHAR (32)
    )
    FILE PATHS 'kifs://vaccine/airport_metadata.csv'
    FORMAT DELIMITED TEXT;
    """)

db.execute_sql("""
    CREATE MATERIALIZED EXTERNAL TABLE "DEMO_Vaccine_Distro"."map_iata_to_country_iso"
    (
       "ident" VARCHAR (8) NOT NULL,
       "type" VARCHAR (16, dict) NOT NULL,
       "name" VARCHAR (128) NOT NULL,
       "elevation_ft" SMALLINT,
       "continent" VARCHAR (2) NOT NULL,
       "iso_country" VARCHAR (2) NOT NULL,
       "iso_region" VARCHAR (8, dict) NOT NULL,
       "municipality" VARCHAR (64),
       "gps_code" VARCHAR (4),
       "iata_code" VARCHAR (4),
       "local_code" VARCHAR (8),
       "coordinates" VARCHAR (64) NOT NULL
    )
    FILE PATHS 'kifs://vaccine/airport-to-country-map.csv'
    FORMAT DELIMITED TEXT;
    """)

{'status_info': {'status': 'ERROR',
  'message': "Error: 'Failed to create external table: DEMO_Vaccine_Distro.map_iata_to_country_iso, Unable to load any files:  (E/Ec:285) (S/SDc:1493); error in Job process'",
  'data_type': 'none',
  'response_time': 0.10203}}

After all the above loads, we should have all the setup we need for graph optimizations. When viewing GAdmin, it should look somewhat like this:

![caption](gadmin.png) 

It would be too complicated to go from every single airport in the US to overseas airports, so it makes sense to consolidate supply within the US into, perhaps into several major regional airports. We start with Washington DC Dulles International Airport for east cost logistics. We create this graph with the following declaration:

In [5]:
    select (dist)/(800*1000/3600) as edge_weight, geo as edge_wktline
            from
            (select  length/(ST_nPoints(geo)-1) as dist, geo
                    from
                    (select ST_length(EDGE_WKTLINE,1) as length, ST_segmentize(EDGE_WKTLINE, 1000000, 1) as geo,
                            from DEMO_Vaccine_Distro.airport_edges 
                            where EDGE_NODE1_ID = 3714 and EDGE_NODE2_ID = 1382
                     )
             ) 

SyntaxError: invalid syntax (<ipython-input-5-605f9e1f6691>, line 1)

In [6]:
    /create/graph {
      "graph_name": "airport",
      "directed_graph": true,
      "nodes": [
        "DEMO_Vaccine_Distro.airport_nodes.NODE_NAME AS NODE_NAME",
        "DEMO_Vaccine_Distro.airport_nodes.NODE_WKTPOINT AS NODE_WKTPOINT",
        "",
        "DEMO_Vaccine_Distro.airport_nodes.NODE_ID AS NODE_ID",
        "DEMO_Vaccine_Distro.airport_nodes.NODE_WKTPOINT AS NODE_WKTPOINT"
      ],
      "edges": [
        "DEMO_Vaccine_Distro.airport_edges.EDGE_ID AS EDGE_ID",
        "DEMO_Vaccine_Distro.airport_edges.EDGE_WKTLINE AS EDGE_WKTLINE",
        "DEMO_Vaccine_Distro.airport_edges.EDGE_DIRECTION AS EDGE_DIRECTION",
        "DEMO_Vaccine_Distro.airport_edges.EDGE_WEIGHT_VALUESPECIFIED AS EDGE_WEIGHT_VALUESPECIFIED"
      ],
      "weights": [],
      "restrictions": [],
      "options": {
           "enable_graph_draw": "true",
           "graph_table": "DEMO_Vaccine_Distro.airport_graph"
        }
    }



SyntaxError: invalid syntax (<ipython-input-6-2d8691eaa3ae>, line 1)

And we then solve the graph with:

    /solve/graph {
      "graph_name": "airport",
      "weights_on_edges": [],
      "restrictions": [],
      "solver_type": "SHORTEST_PATH",
      "source_nodes": [
        "{'IAD: Washington Dulles International Airport'} AS NODE_NAME"
      ],
      "destination_nodes": [],
      "solution_table": "DEMO_Vaccine_Distro.airport_sssp",
      "options": {
        "min_solution_radius": "10000",
        "max_solution_radius": "12000",
        "max_solution_targets": "10",
        "accurate_snaps": "false",
        "output_edge_path": "true",
        "output_wkt_path": "true"
      }
    }


![caption](pix1.png)

![caption](pix2.png)

Finally, we plot out the full route we would use for the round-trip:
    
![caption](path_iad.png)


This is good, but lets continue to refine this. We can add a second departure hub in the US to route vaccine supply from the south, lets set up both IAD (Washington Dulles Airport) and DFW (Dallas Forth Worth Airport.)

We set up the graph:

    /create/graph {
      "graph_name": "airport",
      "directed_graph": true,
      "nodes": [
        "DEMO_Vaccine_Distro.airport_nodes.NODE_NAME AS NODE_NAME",
        "DEMO_Vaccine_Distro.airport_nodes.NODE_WKTPOINT AS NODE_WKTPOINT",
        "",
        "DEMO_Vaccine_Distro.airport_nodes.NODE_ID AS NODE_ID",
        "DEMO_Vaccine_Distro.airport_nodes.NODE_WKTPOINT AS NODE_WKTPOINT"
      ],
      "edges": [
        "DEMO_Vaccine_Distro.airport_edges.EDGE_ID AS EDGE_ID",
        "DEMO_Vaccine_Distro.airport_edges.geodesic AS EDGE_WKTLINE",
        "DEMO_Vaccine_Distro.airport_edges.EDGE_DIRECTION AS EDGE_DIRECTION",
        "DEMO_Vaccine_Distro.airport_edges.weight AS EDGE_WEIGHT_VALUESPECIFIED"
      ],
      "weights": [],
      "restrictions": [],
      "options": {
        "merge_tolerance": "0.0001",
        "use_rtree": "false",
        "min_x": "-180",
        "max_x": "180",
        "min_y": "-90",
        "max_y": "90",
        "recreate": "true",
        "modify": "false",
        "export_create_results": "false",
        "enable_graph_draw": "true",
        "save_persist": "false",
        "sync_db": "false",
        "add_table_monitor": "false",
        "graph_table": "DEMO_Vaccine_Distro.airport_graph",
        "add_turns": "false",
        "turn_angle": "60.0",
        "is_partitioned": "false"
      }
    }

And we match the graph:

    /match/graph {
      "graph_name": "airport",
      "sample_points": [
        "DEMO_Vaccine_Distro.airport_demand.demand_id AS SAMPLE_DEMAND_ID",
        "DEMO_Vaccine_Distro.airport_demand.demand_wkt AS SAMPLE_DEMAND_WKTPOINT",
        "DEMO_Vaccine_Distro.airport_demand.demand_size AS SAMPLE_DEMAND_SIZE",
        "DEMO_Vaccine_Distro.airport_demand.supply_id AS SAMPLE_DEMAND_DEPOT_ID",
        "",
        "DEMO_Vaccine_Distro.airport_supply.supply_id AS SAMPLE_SUPPLY_DEPOT_ID",
        "DEMO_Vaccine_Distro.airport_supply.supply_wkt AS SAMPLE_SUPPLY_WKTPOINT",
        "DEMO_Vaccine_Distro.airport_supply.supply_craft AS SAMPLE_SUPPLY_TRUCK_ID",
        "DEMO_Vaccine_Distro.airport_supply.supply_size AS SAMPLE_SUPPLY_TRUCK_SIZE"
      ],
      "solve_method": "match_supply_demand",
      "solution_table": "DEMO_Vaccine_Distro.airport_msdo_multi1",
      "options": {
        "partial_loading": "true",
        "max_combinations": "10000",
        "aggregated_output": "true",
        "output_tracks": "false",
        "max_trip_cost": "0.0",
        "unit_unloading_cost": "0.0",
        "truck_service_limit": "0.0",
        "max_truck_stops": "0",
        "enable_truck_reuse": "false",
        "left_turn_penalty": "0.0",
        "right_turn_penalty": "0.0",
        "intersection_penalty": "0.0",
        "sharp_turn_penalty": "0.0"
      }
    }

![caption](path_iad_dfw.png)

### Taking this further

Real life is complex. To make predictions or decisions, we create models to represent reality the best we can balancing trade-offs between correctness with complexity. So far, we've made a number of simplifying assumptions, but we can continue to refine the model to introduce more features and address nuances. Some reasonable next steps would be:

* Come up with a more advanced supply model using CDC data but with more nuance around vaccine expirations, expected usage before expiration, etc.
* Come up with a more advanced demand model using OWID data, but with your thoughts on prioriization schems -- do we prioritize the elderly population (more at risk), or prioritize the young (more likely to be outside and spread disease), or prioritize countries with the most deaths, or prioritize the countries with the least access to vaccination?
* Add several more regional US centers, perhps one for the midwest (Chicago ORD) and one for the west (San Francisco SFO)
* Consider that multi-hop drop-offs themselves require time, and thus, consider how many days to expiration are required on vaccines beyond which we should avoid transporting them